In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

housing = fetch_california_housing()
x_train_full, x_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_full, y_train_full, random_state=42)

Exercise 12.

In [3]:
class LayerNormalization(keras.layers.Layer):
    def __init__(self, eps=0.001, **kwargs):
        super().__init__(**kwargs)
        self.eps = eps
    
    def build(self, batch_input_shape):
        self.alpha = self.add_weight(
            name='alpha', 
            shape=batch_input_shape[-1:],
            initializer='ones',
        )
        self.beta = self.add_weight(
            name='beta', 
            shape=batch_input_shape[-1:],
            initializer='zeros',
        )
        super().build(batch_input_shape)
        
    def call(self, x):
        mean, variance = tf.nn.moments(x, axes=-1, keepdims=True)
        return self.alpha * (x - mean) / (tf.sqrt(variance + self.eps)) + self.beta
    
    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'eps': self.eps}

In [4]:
x = x_train.astype(np.float32)

custom_layer_norm = LayerNormalization()
keras_layer_norm = keras.layers.LayerNormalization()

tf.reduce_mean(keras.losses.mean_absolute_error(
    custom_layer_norm(x),
    keras_layer_norm(x),
))

<tf.Tensor: shape=(), dtype=float32, numpy=4.8909556e-08>

In [5]:
random_alpha = np.random.rand(x.shape[-1])
random_beta = np.random.rand(x.shape[-1])

custom_layer_norm.set_weights([random_alpha, random_beta])
keras_layer_norm.set_weights([random_alpha, random_beta])

tf.reduce_mean(keras.losses.mean_absolute_error(
    custom_layer_norm(x),
    keras_layer_norm(x),
))

<tf.Tensor: shape=(), dtype=float32, numpy=1.2229445e-08>